In [2]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 50.5 MB/s 
     |████████████████████████████████| 6.6 MB 36.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 50.0 MB/s 
     |████████████████████████████████| 115 kB 50.9 MB/s 
     |████████████████████████████████| 127 kB 50.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
from transformers import pipeline
from transformers import AutoTokenizer
from datasets import load_dataset

In [22]:
# Drive needs to be mounted for this step, change file path
dataset = load_dataset('csv', data_files={'train': 'drive/MyDrive/TRAIL22/Train_data.csv', 'test': 'drive/MyDrive/TRAIL22/Test_data.csv'})


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("SDG", "labels")
tokenized_dataset = tokenized_dataset.remove_columns(["Unnamed: 0"])
tokenized_dataset = tokenized_dataset.remove_columns(["source"])
tokenized_dataset = tokenized_dataset.remove_columns(["header"])
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=18)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="drive/MyDrive/TRAIL22/test_trainer", evaluation_strategy="epoch")

In [13]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Sep  6 08:51:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1664
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 624


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.875442,0.547748
2,No log,0.961265,0.772973
3,1.769700,0.833902,0.801802


***** Running Evaluation *****
  Num examples = 555
  Batch size = 8
***** Running Evaluation *****
  Num examples = 555
  Batch size = 8
Saving model checkpoint to drive/MyDrive/TRAIL22/test_trainer/checkpoint-500
Configuration saved in drive/MyDrive/TRAIL22/test_trainer/checkpoint-500/config.json
Model weights saved in drive/MyDrive/TRAIL22/test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 555
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=624, training_loss=1.5472924403655224, metrics={'train_runtime': 342.3493, 'train_samples_per_second': 14.582, 'train_steps_per_second': 1.823, 'total_flos': 1313639075414016.0, 'train_loss': 1.5472924403655224, 'epoch': 3.0})

In [16]:
model.save_pretrained("drive/MyDrive/TRAIL22/saved_model")

Configuration saved in drive/MyDrive/TRAIL22/saved_model/config.json
Model weights saved in drive/MyDrive/TRAIL22/saved_model/pytorch_model.bin


In [17]:
mod=AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/TRAIL22/saved_model")

loading configuration file drive/MyDrive/TRAIL22/saved_model/config.json
Model config BertConfig {
  "_name_or_path": "drive/MyDrive/TRAIL22/saved_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 1

In [18]:
classifier = pipeline("text-classification", model=mod, tokenizer=tokenizer)

In [19]:
classifier("We will work to restore hunger in developping countries.")

[{'label': 'LABEL_2', 'score': 0.8761239051818848}]

In [20]:
classifier("I will teach the students about SDG 2.")

[{'label': 'LABEL_4', 'score': 0.8264462351799011}]

In [21]:
classifier("I don't think I can afford the gas bill this month.")

[{'label': 'LABEL_12', 'score': 0.41769787669181824}]

In [ ]:
classifier("I am not hungry.")

[{'label': 'LABEL_2', 'score': 0.7281983494758606}]